In [14]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign

Useful Functions!

In [15]:
# read_response checks if the response of the server is valid JSON

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

# get_measurement_unit returns the measurement unit of a given pollutant

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]

# get_province_shape returns a GeoDataFrame containing the geometries of the provinces
   
def get_province_shape():
    t=requests.get(url="http://127.0.0.1:5000/api/province_shape")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry_province'] = gdf['geometry_province'].apply(wkt.loads)
    gdf.set_geometry('geometry_province', inplace=True)
    return gdf

# list_sensors_and_pollutants gets the list of sensor and relative pollutants for a given station

def list_sensors_and_pollutants(list_stations):
    t=requests.post(url="http://127.0.0.1:5000/api/sensors_and_pollutants", json={"var_id_stazione": list_stations}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data

# get_station_location returns a GeoDataFrame containing the points of the stations

def get_station_location():
    t=requests.get(url="http://127.0.0.1:5000/api/station_location")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)
    return gdf

############################################# FUNCTION TO POPULATE THE DROPDOWN #############################################

# pollutant_dropdown populates the dropdown with the pollutatnts that have at least one recorded value

def pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

# all_pollutant_dropdown populates the dropdown with all the pollutants, even those that have no recorded values

def all_pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/all_pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

############################################# FUNCTION FOR MAP VISUALIZATION #############################################

# df_to_dash_table converts a DataFrame to a Dash HTML table for display in the popup of EU_DV_1

def df_to_dash_table(df):
    return html.Table([
        html.Thead(html.Tr([html.Th(col) for col in df.columns])),
        html.Tbody([
            html.Tr([html.Td(df.iloc[i][col]) for col in df.columns]) for i in range(len(df))
        ])
    ], style={"maxWidth": "300px", "fontSize": "12px"})

Definition of variables for visualization

In [25]:
# Initialize the Dash app
app = Dash(__name__)

############################################# EU_DV_2 ###############################################

import ipywidgets as widgets
from IPython.display import display
import requests
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from dash import Dash, html, dcc, Input, Output, State
import pandas as pd
import plotly.express as px
import requests
from datetime import date

# Utility Function to Read API Responses

def read_response(response):
    try:
        return response.json()
    except requests.exceptions.JSONDecodeError:
        print("Invalid JSON response!")
        print("Response content:", response.text)
        return None
    
# Get List of Available Provinces

def get_provinces_list():
    response = requests.get("http://127.0.0.1:5000/api/provinces")
    return read_response(response) or []


# Get List of Available Pollutants

def get_pollutants_list():
    response = requests.get("http://127.0.0.1:5000/api/pollutants")
    return read_response(response) or []

# Get startdate and enddata possible

def get_data_dict():
    response = requests.get("http://127.0.0.1:5000/api/data")
    return read_response(response) or []

# Slider soglia
slider_soglia = widgets.IntSlider(
    value=0,
    min=0,
    max=80,
    step=1,
    description='Soglia:',
    continuous_update=False
)

############################################# Layout EU_DV_2 #############################################

app = Dash(__name__)

app.layout = html.Div([
    html.H1('Bugs_project: Air quality analysis'),
    html.P('Description of the dashboard functionalities'),

    html.Div([
        html.H2('EU_DV_2 Threshold definition – time-series'),
        html.P("Select a pollutant, province, date range, and threshold to visualize which days exceeded it."),

        dcc.Dropdown(
            id='all-pollutant-dropdown_EU_DV_2',
            options=get_pollutants_list(),
            placeholder="Select a pollutant",
            style={'margin-top': 10}
        ),

        dcc.Dropdown(
            id='province-dropdown_EU_DV_2',
            options=get_provinces_list(),
            placeholder="Select a province",
            style={'margin-top': 10}
        ),

        dcc.DatePickerRange(
            id='date-range-picker_EU_DV_2',
            min_date_allowed=date(2020, 1, 1),
            max_date_allowed=date(2025, 12, 31),
            start_date_placeholder_text="Start Date",
            end_date_placeholder_text="End Date",
            display_format='YYYY-MM-DD',
            style={'margin-top': 10}
        ),

        html.Label("Threshold (µg/m³):", style={'margin-top': 20}),
        dcc.Slider(
            id='threshold-slider_EU_DV_2',
            min=0,
            max=80,
            step=1,
            value=50,
            marks={i: str(i) for i in range(0, 201, 20)},  # Mostra solo ogni 20 µg/m³
            tooltip={"placement": "bottom", "always_visible": True},
            included=False,
            updatemode='drag',  # aggiorna mentre trascini
            style={'margin-top': '15px', 'margin-bottom': '30px'}
        ),

        html.Div([
            html.Button(
                'Visualize threshold exceedance',
                id='button_EU_DV_2',
                n_clicks=0,
                style={'margin-top': 20, 'margin-bottom': 20}
            )
        ], style={'textAlign': 'center'}),

        html.Div(id='exceedance-percent_EU_DV_2', style={'textAlign': 'center'}),
        dcc.Graph(id='histogram-output_EU_DV_2')
    ])
])

############################################# Callback EU_DV_2 #############################################


@app.callback(
    Output('histogram-output_EU_DV_2', 'figure'),
    Output('exceedance-percent_EU_DV_2', 'children'),
    Input('button_EU_DV_2', 'n_clicks'),
    State('all-pollutant-dropdown_EU_DV_2', 'value'),
    State('province-dropdown_EU_DV_2', 'value'),
    State('date-range-picker_EU_DV_2', 'start_date'),
    State('date-range-picker_EU_DV_2', 'end_date'),
    State('threshold-slider_EU_DV_2', 'value')
)
def update_time_series(n_clicks, pollutant, province, start_date, end_date, threshold):
    if not all([pollutant, province, start_date, end_date, threshold]):
        return {}, "⚠️ Please select all inputs."

    payload = {
        "var_pollutant": pollutant,
        "var_start_date": f"{start_date} 00:00:00",
        "var_end_date": f"{end_date} 23:59:59",
        "var_province": province,
        "var_threshold": threshold
    }

    try:
        response = requests.post("http://127.0.0.1:5000/api/EU_DV_2", json=payload)
        response.raise_for_status()
        data = response.json()
        if not data:
            return {}, "No exceedance data found."

        df = pd.DataFrame(data)
        df['data'] = pd.to_datetime(df['data'])
        df = df.sort_values('data')

        # Raggruppa per giorno
        df_grouped = df.groupby(df['data'].dt.date)['valore'].mean().reset_index()
        df_grouped.columns = ['Data', 'Valore medio']

        # Calcolo percentuale
        unique_days = df['data'].dt.date.nunique()
        total_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days + 1
        percent = (unique_days / total_days) * 100

        fig = px.bar(df_grouped, x='Data', y='Valore medio', title="Days exceeding threshold")
        fig.add_hline(y=threshold, line_dash="dash", line_color="blue", annotation_text=f"Threshold = {threshold}")

        return fig, f"✅ Exceeded on {unique_days} out of {total_days} days ({percent:.1f}%)"

    except Exception as e:
        return {}, f"❌ Error: {str(e)}"

if __name__ == '__main__':
    print("Starting Dashboard...")
    app.run(port=8089, debug=True)




TypeError: The `dcc.Slider` component (version 3.0.4) with the ID "threshold-slider_EU_DV_2" received an unexpected keyword argument: `style`
Allowed arguments: className, disabled, dots, drag_value, id, included, marks, max, min, persisted_props, persistence, persistence_type, step, tooltip, updatemode, value, vertical, verticalHeight